In [4]:
import numpy as np
import pandas as pd
import os
import json
import openai
import sys
import importlib
import json_repair as jr
from dotenv import load_dotenv
import anthropic
import random
from google import genai

import creators_sct as cr
#import judges_sct as jg


In [6]:
#importlib.reload(jg)
importlib.reload(cr)

<module 'creators_sct' from '/home/isaac/Documents/VUMC/reasyn25/creator/creators_sct.py'>

In [3]:
import os
import sys
cwd = os.getcwd()
judge_path = os.path.join(cwd, "judge")
creator_path = os.path.join(cwd, "creator")

sys.path.append(judge_path)
sys.path.append(creator_path)

In [18]:
input = ["psychiatry", True, True]
goob = cr.gem25p_creator_synth_sct(input)

In [19]:
goob

array(['A 22-year-old male university student presents for a psychiatric evaluation due to a 6-month history of persistent low mood, anhedonia, and significant fatigue. He reports difficulty concentrating on his studies, which has caused his grades to drop. He was started on escitalopram by his primary care physician two months ago, titrating up to the maximum dose of 20mg daily, but has experienced minimal improvement. Routine lab work, including a TSH, CBC, and comprehensive metabolic panel, is within normal limits.',
       'Switch antidepressant to an SNRI like venlafaxine.',
       "On further questioning about his past, he mentions a period during his first year of university where he felt 'on top of the world,' slept only 3 hours a night for a week, and impulsively bought a non-refundable plane ticket for a trip he never took.",
       'This subsequent finding strongly suggests a past hypomanic episode, which would change the diagnosis from Major Depressive Disorder to Bipolar I